## Graph and session example

Code Created by Luis Enrique Acevedo Galicia

Date: 2019-27-03

Here, I present a simple and easy way to create a graph with its own session. Just as TensorFlow does.

# The General Operation

The operation is a a node in a "Graph".

The Operation class is inherited by other classes that compute the specific operations, such as adding or matrix multiplication.

In [1]:
import numpy as np

In [2]:
class TheOperation():
    
    #Here, we initilize any operation
    def __init__(self, Nodes_Input = []):
        
        self.Nodes_Input = Nodes_Input #This is the list of inputs
        self.Nodes_Ouput = [] # This is the list of empty outputs
        
        for Nodes in Nodes_Input:
            Nodes.Nodes_Ouput.append(self) #for inputs nodes, we append the operation
    
        _main_graph.Operation.append(self)
    
    def computing(self):
        pass #provisional placeholder, the actual operation will be inherit here.

# Operations

Here, all the operations will be defined as a class

# Add +

In [3]:
class add(TheOperation):
    
    def __init__(self, a,b):
        super().__init__([a,b]) #we assign the inputs of the operation
        
    def computing(self, a,b): #this is the actual operation
        self.Inputs = [a,b]
        return a + b

We can re-use the same code to create the rest of the operations

# Multiplication *

In [4]:
class multiply(TheOperation):
    
    def __init__(self, a,b):
        super().__init__([a,b]) #we assign the inputs of the operation
        
    def computing(self, a,b): #this is the actual operation
        self.Inputs = [a,b]
        return a * b

# Multiplication (Matrix) .*

In [5]:
class multiplyM(TheOperation):
    
    def __init__(self, a,b):
        super().__init__([a,b]) #we assign the inputs of the operation
        
    def computing(self, a,b): #this is the actual operation
        self.Inputs = [a,b]
        return a.dot(b)

# The placeholders

The placeholder is a empty node that will need a value for compute the outputs in the Graph.


In [6]:
class Theplaceholder():
    
    def __init__(self):
        self.Nodes_Ouput = [] #at the beginning it is an empty list
        _main_graph.Placeholders.append(self) #use a global variable to access to the graph

# The Variables

Here, all variables are defined as a changeable value of the Graph.

In [7]:
class TheVariable():
    
    def __init__(self, value_initial = None): #the variable is initialize as empty
        self.value = value_initial
        self.Nodes_Ouput = []
        _main_graph.Variables.append(self) #use a global variable to access to the graph

# Order the operations, variables and placeholders

This function creates a list of all operations, variables and placeholders in correct order to the be computed.

In [8]:
def Order_operation(operation):
    
    Ordered_nodes = []
    
    def order(Operation_nodes):
        if isinstance(Operation_nodes, TheOperation): #If the node is part of an operation
            for Nodes_input in Operation_nodes.Nodes_Input:
                order(Nodes_input)
        Ordered_nodes.append(Operation_nodes)
    
    order(operation)
    return Ordered_nodes

# The Graph

Here is wehre the Graph is defined, to then perform computations in a session.

In [9]:
class TheGraph():
    
    def __init__(self):
        
        self.Operation = []
        self.Variables = []
        self.Placeholders = []
    
    def Default(self):
        
        global _main_graph #definition of global variable
        _main_graph = self

# The session

In [10]:
class TheSession():
    
    def run(self, operation, Placeholder_dict = {}): #Placeholder_dict is the dictionary defining the placeholders
        
        #Create the list of operations, variables and placeholders
        
        Ordered_nodes = Order_operation(operation)
        
        for Nodes in Ordered_nodes:
            if type(Nodes) == TheVariable:
                Nodes.output = Nodes.value #the variables values  
            elif type(Nodes) == Theplaceholder:
                Nodes.output = Placeholder_dict[Nodes] #feed values to placeholders
            else: #perform the operations
                Nodes.inputs = [Nodes_Input.output for Nodes_Input in Nodes.Nodes_Input]
                Nodes.output = Nodes.computing(*Nodes.inputs) #The * means that we do not know the number of parameters
            if type(Nodes.output) == list:
                Nodes.output = np.array(Nodes.output) #Convert to np array
                
        return operation.output

# Perform the operation

## Create a Graph

In [11]:
Graph_test = TheGraph()
Graph_test.Default()

## Define variables, placeholders and operations

In [12]:
A1 = TheVariable(20) #value 20
Ph = Theplaceholder() #empty
OP1 = multiply(A1,Ph) #multiplication
A2 = TheVariable(5) #value 5
OP2 = add(A2,OP1) # addition


# Define the session and compute

First test with scalar values

In [15]:
sess = TheSession()

Results = sess.run(operation = OP2, Placeholder_dict = {Ph:50})

In [16]:
print(Results)


1005


Second test with Matrix

In [27]:

#Define the graph
G2 = TheGraph()
G2.Default()

#Define Varibales, placeholders and operations
B1 = TheVariable([[15,21],[300,450]])
B2 = TheVariable([3,3])
PH2 = Theplaceholder()
OP3 = multiplyM(B1,PH2)
OP4 = add(OP3,B2)

#Define the session
sess2 = TheSession()
Results2 = sess2.run(operation = OP4, Placeholder_dict = {PH2:[[15,21],[300,450]]})
print(Results2)



[[  6528   9768]
 [139503 208803]]
